In [2]:
import json

# Helper function to convert latitude and longitude to cartesian coordinates
def convert_to_grid_coords(lat, lng, lat_min, lat_max, lng_min, lng_max, grid_size=1000, depot_position=(500, 500)):
    # Normalizing latitude and longitude
    lat_norm = (lat - lat_min) / (lat_max - lat_min)
    lng_norm = (lng - lng_min) / (lng_max - lng_min)

    # Scaling to the grid size
    x = int(lng_norm * grid_size)
    y = int(lat_norm * grid_size)

    # Translating so that depot is at the specified position
    x += (depot_position[0] - int(grid_size / 2))
    y += (depot_position[1] - int(grid_size / 2))

    return x, y

# Path to the JSON file
file_path_json = '/Users/cstenico/Documents/shire/tcc-mba/loggibud/data/025/vrppd-instances-1.0/train/pa-0/cvrp-0-pa-0.json'


# Loading the JSON data
with open(file_path_json, 'r') as file:
    json_data = json.load(file)

# Extracting depot data
depot = json_data['depot']

partial_demands = json_data['demands']

# Finding the min and max latitude and longitude values from the JSON data
latitudes = [demand['point']['lat'] for demand in partial_demands] + [depot['lat']]
longitudes = [demand['point']['lng'] for demand in partial_demands] + [depot['lng']]
lat_min, lat_max = min(latitudes), max(latitudes)
lng_min, lng_max = min(longitudes), max(longitudes)

# Converting the depot and demands to VRP format with the new coordinate system
depot_coords = convert_to_grid_coords(depot['lat'], depot['lng'], lat_min, lat_max, lng_min, lng_max)
demands = [f"1\t0"]  # Demand at depot is zero

# Preparing nodes with the new coordinate system
nodes = [f"1\t{depot_coords[0]}\t{depot_coords[1]}"]  # Depot as the first node
for i, demand in enumerate(partial_demands, start=2):
    node_coords = convert_to_grid_coords(demand['point']['lat'], demand['point']['lng'], lat_min, lat_max, lng_min, lng_max)
    nodes.append(f"{i}\t{node_coords[0]}\t{node_coords[1]}")
    demands.append(f"{i}\t{demand['size'] if demand['type'] == 'PICKUP' else -demand['size']}")

# Constructing the VRP file content
vrp_content = f"NAME : {json_data['name']}\n"
vrp_content += "TYPE : CVRP\n"
vrp_content += f"DIMENSION : {len(nodes)}\n"
vrp_content += "EDGE_WEIGHT_TYPE : EUC_2D\n"
vrp_content += f"CAPACITY : {json_data['vehicle_capacity']}\n"
vrp_content += "NODE_COORD_SECTION\n"
vrp_content += '\n'.join(nodes)
vrp_content += "\nDEMAND_SECTION\n"
vrp_content += '\n'.join(demands)
vrp_content += "\nDEPOT_SECTION\n1\n-1\nEOF"


In [3]:
vrp_content

'NAME : cvrp-0-pa-0\nTYPE : CVRP\nDIMENSION : 348\nEDGE_WEIGHT_TYPE : EUC_2D\nCAPACITY : 180\nNODE_COORD_SECTION\n1\t489\t437\n2\t494\t455\n3\t497\t459\n4\t504\t465\n5\t501\t448\n6\t513\t462\n7\t499\t460\n8\t490\t454\n9\t506\t445\n10\t510\t465\n11\t497\t449\n12\t506\t460\n13\t494\t455\n14\t498\t465\n15\t507\t471\n16\t499\t462\n17\t497\t459\n18\t499\t457\n19\t501\t458\n20\t503\t459\n21\t494\t463\n22\t490\t464\n23\t512\t452\n24\t496\t465\n25\t494\t462\n26\t508\t461\n27\t502\t447\n28\t513\t464\n29\t507\t459\n30\t503\t453\n31\t494\t462\n32\t509\t474\n33\t496\t457\n34\t494\t464\n35\t41\t422\n36\t57\t429\n37\t51\t375\n38\t32\t430\n39\t37\t444\n40\t39\t447\n41\t48\t421\n42\t57\t421\n43\t40\t411\n44\t56\t423\n45\t51\t438\n46\t37\t406\n47\t33\t428\n48\t55\t390\n49\t46\t421\n50\t45\t427\n51\t54\t424\n52\t0\t482\n53\t49\t412\n54\t475\t447\n55\t472\t460\n56\t481\t442\n57\t468\t433\n58\t467\t454\n59\t477\t436\n60\t244\t438\n61\t247\t595\n62\t271\t384\n63\t224\t574\n64\t255\t415\n65\t284\t389\n66\t1

In [4]:
output_file_path = 'instances/PA_0/PA.vrp'

# Saving the VRP content to a file
with open(output_file_path, 'w') as file:
    file.write(vrp_content)

In [19]:
from vrp.data_utils import InstanceBlueprint

blue_print = InstanceBlueprint(nb_customers=len(partial_demands) - 1,
depot_position='R',
customer_position='R',
nb_customer_cluster=None,
demand_type='inter',
demand_min=min(demand['size'] for demand in partial_demands),
demand_max=max(demand['size'] for demand in partial_demands),
capacity=json_data['vehicle_capacity'],
grid_size=1000)

In [20]:
blue_print.__dict__


{'nb_customers': 99,
 'depot_position': 'R',
 'customer_position': 'R',
 'nb_customers_cluster': None,
 'demand_type': 'inter',
 'demand_min': 1,
 'demand_max': 10,
 'capacity': 180,
 'grid_size': 1000}